In [ ]:
import json
import sys
import osmnx as ox
from ast import literal_eval
from os.path import join
from setup_paths import paths
sys.path.append(paths["project_dir"])

In [ ]:
def get_yearly_added(yearly):
    yearly_added = {year: set(yearly[year])-set(yearly[year-1]) for year in range(1,51)}
    segment_year = {segment: year for year, year_segments in yearly_added.items() for segment in year_segments}
    
    return yearly_added, segment_year

In [ ]:
def load_orderings(base, new, base_mode="LPGreedy", new_mode="TA"):
    with open(join(paths["output_folder"], base, f"{base}_order_{base_mode}.json"), "r") as f:
        base_order = json.load(f)
    base_segment_place = {seg: idx+1 for idx, seg in enumerate(base_order)}
    
    with open(join(paths["output_folder"], new, f"{new}_order_yearly_{new_mode}.json"), "r") as f:
        new_yearly = json.load(f)
    new_yearly = {int(k[4:]): v for k, v in new_yearly.items()}
    new_yearly[0] = []
    
    new_yearly_added, new_segment_year = get_yearly_added(new_yearly)
    
    return base_order, base_segment_place, new_yearly_added

In [ ]:
def segment_ordering(base, new):
    base_order, base_segment_place, new_yearly_added = load_orderings(base, new)
    
    new_segment_order = []

    for year in range(1, 51):
        new_segment_place_peryear = {seg: base_segment_place[seg] for seg in new_yearly_added[year]}
        year_reordered = list(sorted(new_segment_place_peryear, key=new_segment_place_peryear.get))
        new_segment_order.extend(year_reordered)
    
    unused_segments = set(base_order) - set(new_segment_order)
    
    return new_segment_order, unused_segments

In [ ]:
saves = {v: v for v in [f"cph_dtu_noisy_demand_{i}" for i in range(1, 11)] + [f"cph_dtu_noisy_costs_{i}" for i in range(1, 11)] + [f"cph_dtu_noisy_speeds_{i}" for i in range(1, 11)]}

In [ ]:
g = ox.load_graphml(join(paths["input_folder"], "cph_dtu", "cph_dtu.graphml"))
for u, v, d in g.edges(data=True):
    d["ex_inf"] = literal_eval(d["ex_inf"])
    d["seg_id"] = literal_eval(d["seg_id"])

In [ ]:
for new_save, base_save in saves.items():
    n_ordering, u_segments = segment_ordering(base_save, new_save)
    
    with open(join(paths["output_folder"], new_save, f"{new_save}_order_TA.json"), "w") as f:
      json.dump(n_ordering, f)
        
    add_blocked_edges = []
    for u, v, d in g.edges(data=True):
        if d["seg_id"] in u_segments:
            add_blocked_edges.append((u, v))
    with open(join(paths["input_folder"], new_save, f"{new_save}_add_blocked_edges_TA.json"), "w") as f:
          json.dump(add_blocked_edges, f)